In [27]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime


In [28]:
os.getcwd()

'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/Zalreadydone'

In [29]:
seasons_teams = pd.read_csv("/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/teams in top division by season, 2013-24.csv")

In [30]:
import ast

def parse_string_to_list(input_string):
    try:
        # Use ast.literal_eval to safely evaluate the string as a Python literal
        result_list = ast.literal_eval(input_string)
        return result_list
    except (SyntaxError, ValueError) as e:
        # Handle potential errors in parsing the string
        print(f"Error: {e}")
        return None

In [31]:
def grab_transfer_pagesoup(input_url):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    pageTree = requests.get (input_url, headers = headers)
    pageSoup_club = BeautifulSoup (pageTree.content, 'html.parser')
    
    return pageSoup_club

In [32]:
def find_url_and_code(input_club_name, input_season_year):
    search = f"{input_club_name} Squad {input_season_year} Transfermarkt"
    url = 'https://www.google.com/search'

    headers = {'Accept' : '*/*', 'Accept-Language': 'en-US,en;q=0.5','User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82',
}
    parameters = {'q': search}

    content = requests.get(url, headers = headers, params = parameters).text
    soup = BeautifulSoup(content, 'html.parser')

    search = soup.find(id = 'search')
    first_link = search.find('a')

    url_tosplit = (first_link['href'])

    name_club = url_tosplit.split('/')[3]
    code_club = url_tosplit.split('/')[6]

    #url_to_return = f"https://www.transfermarkt.us/{name_club}/alletransfers/verein/{code_club}"

    return name_club, code_club

    

    

In [33]:
#youre gonna have the team name and the season 

#the function needs to have those as inputs and spit out the URL 

def generate_url(input_club_name, input_code, input_szn):
    return f"https://www.transfermarkt.us/{input_club_name}/kader/verein/{input_code}/plus/1/galerie/0?saison_id={input_szn}"


In [34]:
def grab_squad_table(input_pagesoup):
    #NAMES 
    Players_set = input_pagesoup.find_all("td", {"class": "hauptlink"})
    #AGE NATIONALITY CURRENT CLUB 
    zentriert_set = input_pagesoup.find_all("td", {"class": "zentriert"})
    #value
    value_set = input_pagesoup.find_all("td", {"class": "rechts hauptlink"})
    #POSITION / TEAMS PLAYED FOR 
    table_set = input_pagesoup.find_all("table", {"class": "inline-table"})

    players_names_array = []
    ages_array = []
    positions_array = []
    nationality_array = []
    market_value_array = [] 

    #age_index = 0
    #nationality_index = 0
    zentriert_Index = 0

    for j in range(0, len(value_set)):
        player_index = (j * 2) #at 1 this is 2
        zentriert_Index = (j * 8) #at 1 this is 8
        #table_index = (j * 4)
    #name
        players_names_array.append(Players_set[player_index].text.strip().encode().decode("utf-8"))
    #DOB
        ages_array.append(zentriert_set[(zentriert_Index+1)].text.strip().encode().decode("utf-8"))
    #pos
        positions_array.append(table_set[j].find_all("td")[2].text.strip().encode().decode("utf-8"))
    #natl
        nationality = str(zentriert_set[(zentriert_Index + 2)].find_all("img", {"class": "flaggenrahmen"})).split(' ')
        if(nationality[0] == '[]'):
            nationality_array.append('None')
        elif (len(nationality) == 5):
            nationality_array.append(nationality[4][7:-4])
        else:
            issit = False
            words_array = []
            words_array.append(nationality[1].split('alt="')[1])
            e = 2
            while(issit == False):
                #check if it starts with class
                if(nationality[e].startswith('class')):
                    #end it 
                    issit = True
                    #print(nationality[e] + 'false')

                else:
                    #add it 
                    words_array.append(nationality[e].split('"')[0]) #was "/>]"
                    #print(nationality[e])
                    e += 1
                    
            final_name = ' '.join(words_array)
            nationality_array.append(final_name)
    #value
        market_value_array.append(value_set[j].text.strip().encode().decode("utf-8"))

    df_to_return = pd.DataFrame(
        {"Name": players_names_array,
        "Age (that year)": ages_array,
        "Nationality": nationality_array,
        "Position": positions_array,
        "Market Value": market_value_array})

    return df_to_return



    

# Running

In [35]:
league_names = seasons_teams['League'].unique()
seasons = list(range(2013, 2024))

In [36]:
grand_dataframe_transfermarkt = pd.DataFrame()

In [40]:
#loop through the leagues 
input_league = league_names[7]
league_dataframe = pd.DataFrame()
for i in range(0, len(seasons)):

#for i in range(0, 1):
    input_season = seasons[i]
    
    series1 = seasons_teams[(seasons_teams['League'] == f"{input_league}") & (seasons_teams['Season'] == input_season)]['Teams'].values
    
    input_string = series1[0]
    
    result_list_teams_season = parse_string_to_list(input_string)
    
    #loop through the teams 
    for j in range(0, len(result_list_teams_season)):
        team = result_list_teams_season[j]

        if(team == 'Vitesse'):
            url = generate_url('vitesse-arnhem2', 499, input_season)
        elif(team == 'Vitória'):
            url = generate_url('vitoria-guimaraes-sc', 2420, input_season)
        elif(team == 'Nacional'):
            url = generate_url('cd-nacional', 982, input_season)
        elif(team == 'Marítimo'):
            url = generate_url('cs-maritimo', 1301, input_season)
        else:
            url_and_code = find_url_and_code(team, input_season)
            url = generate_url(url_and_code[0], url_and_code[1], input_season)
        
        
        squad_page = grab_transfer_pagesoup(url)
        
        squad_table = grab_squad_table(squad_page)

        squad_table['Season'] = input_season
        squad_table['League'] = input_league
        squad_table['Team'] = team

        league_dataframe = pd.concat([league_dataframe, squad_table], ignore_index=True)

        #maybe make sure league_dataframe['Team'].value_counts() == the size of the league you expect 

In [59]:
for i in range(2013, 2024):
    subset_df = league_dataframe[league_dataframe['Season'] == i]
    print(i, len(subset_df['Team'].value_counts()))

2013 16
2014 18
2015 18
2016 18
2017 18
2018 18
2019 18
2020 18
2021 18
2022 18
2023 18


In [62]:
grand_dataframe_transfermarkt = pd.concat([grand_dataframe_transfermarkt, league_dataframe], ignore_index=True)


In [63]:
grand_dataframe_transfermarkt['League'].value_counts()

League
Portuguese First Division    6900
Name: count, dtype: int64

## small data changes

In [64]:
for i in range(0, len(grand_dataframe_transfermarkt)):
    this_row_team = grand_dataframe_transfermarkt.at[i, 'Team']
    if(this_row_team == 'B-SAD'):
        grand_dataframe_transfermarkt.at[i, 'Team'] = 'Belenenses'

In [65]:
for i in range(0, len(grand_dataframe_transfermarkt)):
    this_row_team = grand_dataframe_transfermarkt.at[i, 'Team']
    if(this_row_team == 'Gil Vicente FC'):
        grand_dataframe_transfermarkt.at[i, 'Team'] = 'Gil Vicente'

In [66]:
for i in range(0, len(grand_dataframe_transfermarkt)):
    this_row_team = grand_dataframe_transfermarkt.at[i, 'Team']
    if(this_row_team == 'Gaziantep FK'):
        grand_dataframe_transfermarkt.at[i, 'Team'] = 'Gaziantepspor'

In [67]:
for i in range(0, len(grand_dataframe_transfermarkt)):
    this_row_team = grand_dataframe_transfermarkt.at[i, 'Team']
    if(this_row_team == 'Erzurum BB'):
        grand_dataframe_transfermarkt.at[i, 'Team'] = 'BB Erzurumspor'

In [68]:
grand_dataframe_transfermarkt.to_csv('2nd_LIGAPORTUGAL_transfermarkt_league_data_checking.csv', index=False)


## Debugging


In [41]:
for input_season in range(2013, 2024):
    series1 = seasons_teams[(seasons_teams['League'] == "Portuguese First Division") & (seasons_teams['Season'] == input_season)]['Teams'].values
    print(input_season, series1[0])

2013 ['Benfica', 'Sporting CP', 'Porto', 'Estoril', 'Nacional', 'Marítimo', 'Vitória Setúbal', 'Académica', 'Braga', 'Vitória', 'Rio Ave', 'Arouca', 'Gil Vicente', 'Belenenses', 'Paços', 'Olhanense']
2014 ['Benfica', 'Porto', 'Sporting CP', 'Braga', 'Vitória', 'Belenenses', 'Nacional', 'Paços', 'Marítimo', 'Rio Ave', 'Moreirense', 'Estoril', 'Boavista', 'Vitória Setúbal', 'Académica', 'Arouca', 'Gil Vicente', 'Penafiel']
2015 ['Benfica', 'Sporting CP', 'Porto', 'Braga', 'Arouca', 'Rio Ave', 'Paços', 'Estoril', 'Belenenses', 'Vitória', 'Nacional', 'Moreirense', 'Marítimo', 'Boavista', 'Vitória Setúbal', 'Tondela', 'União', 'Académica']
2016 ['Benfica', 'Porto', 'Sporting CP', 'Vitória', 'Braga', 'Marítimo', 'Rio Ave', 'Feirense', 'Boavista', 'Estoril', 'Chaves', 'Vitória Setúbal', 'Paços', 'Belenenses', 'Moreirense', 'Tondela', 'Arouca', 'Nacional']
2017 ['Porto', 'Benfica', 'Sporting CP', 'Braga', 'Rio Ave', 'Chaves', 'Marítimo', 'Boavista', 'Vitória', 'Portimonense', 'Tondela', 'Belen

### grab url & code

In [121]:
seasons_teams

'Vitesse Squad 2020 Transfermarkt'

In [120]:
input_club_name = 'Vitesse'
input_season_year = 2020

search = f"{input_club_name} Squad {input_season_year} Transfermarkt"
url = 'https://www.google.com/search'

headers = {'Accept' : '*/*', 'Accept-Language': 'en-US,en;q=0.5','User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82',
}
parameters = {'q': search}

content = requests.get(url, headers = headers, params = parameters).text
soup = BeautifulSoup(content, 'html.parser')

# search = soup.find(id = 'search')
# first_link = search.find('a')

# url_tosplit = (first_link['href'])

# name_club = url_tosplit.split('/')[3]
# code_club = url_tosplit.split('/')[6]

### grab squad

In [98]:
#NAMES 
Players_set = squad_page.find_all("td", {"class": "hauptlink"})
#AGE NATIONALITY CURRENT CLUB 
zentriert_set = squad_page.find_all("td", {"class": "zentriert"})
#value
value_set = squad_page.find_all("td", {"class": "rechts hauptlink"})
#POSITION / TEAMS PLAYED FOR 
table_set = squad_page.find_all("table", {"class": "inline-table"})



In [99]:
players_names_array = []
ages_array = []
positions_array = []
nationality_array = []
market_value_array = [] 

age_index = 0
#nationality_index = 0
zentriert_Index = 0

for j in range(0, len(value_set)):
    player_index = (j * 2) #at 1 this is 2
    zentriert_Index = (j * 8) #at 1 this is 8
    #table_index = (j * 4)
#name
    players_names_array.append(Players_set[player_index].text.strip().encode().decode("utf-8"))
#DOB
    ages_array.append(zentriert_set[(zentriert_Index+1)].text.strip().encode().decode("utf-8"))
#pos
    positions_array.append(table_set[j].find_all("td")[2].text.strip().encode().decode("utf-8"))
#natl
    nationality = str(zentriert_set[(zentriert_Index + 2)].find_all("img", {"class": "flaggenrahmen"})).split(' ')
    if(nationality[0] == '[]'):
        nationality_array.append('None')
    elif (len(nationality) == 5):
        nationality_array.append(nationality[4][7:-4])
    else:
        issit = False
        words_array = []
        words_array.append(nationality[1].split('alt="')[1])
        e = 2
        while(issit == False):
            #check if it starts with class
            if(nationality[e].startswith('class')):
                #end it 
                issit = True
                #print(nationality[e] + 'false')

            else:
                #add it 
                words_array.append(nationality[e].split('"')[0]) #was "/>]"
                #print(nationality[e])
                e += 1
                
        final_name = ' '.join(words_array)
        nationality_array.append(final_name)
#value
    market_value_array.append(value_set[j].text.strip().encode().decode("utf-8"))

df_to_return = pd.DataFrame(
    {"Name": players_names_array,
    "Age (that year)": ages_array,
    "Nationality": nationality_array,
    "Position": positions_array,
    "Market Value": market_value_array})



In [15]:
# seasons_teams.at[23, 'Teams']
# seasons_teams[(seasons_teams['League'] == 'Serie A') & (seasons_teams['Season'] == 2014)]#['Teams']